In [8]:
import numpy as np
from tabulate import tabulate
import random

def theShip(D):
    options = []
    ship = np.full((D, D), int(0), dtype = object)
    X, Y = random.randrange(D), random.randrange(D)
    
    markOpen(ship, X, Y, D, options)
    while len(options) >= 1:
        rand = random.randrange(len(options))
        X, Y = options[rand]
        options = markOpen(ship, X, Y, D, options)

    #print(tabulate(ship))
    #print(options)  

    deadends = []
    for X in range(D):
        for Y in range(D):
            if isinstance(ship[X][Y], int):
                ship[X][Y] = 0
            if isinstance(ship[X][Y], str):
                deadend = 0
                if X - 1 >= 0:
                    if ship[X - 1][Y] == 'X':
                        deadend += 1
                if X + 1 < D:
                    if ship[X + 1][Y] == 'X':
                        deadend += 1
                if Y - 1 >= 0:
                    if ship[X][Y - 1] == 'X':
                        deadend += 1
                if Y + 1 < D:
                    if ship[X][Y + 1] == 'X':
                        deadend += 1
                        
                if deadend == 1:
                    deadends.append(tuple([X, Y]))
    random.shuffle(deadends)
         
    print(tabulate(ship))
    print(deadends)

    for i in range(int(len(deadends) / 2)):
        X, Y = deadends[i]
        randselect = []
        randnum = 0
        if Y - 1 >= 0:
            if isinstance(ship[X][Y - 1], int):
                randselect.append(ship[X][Y - 1])
                randnum += 1
        if X - 1 >= 0:
            if isinstance(ship[X - 1][Y], int):
                randselect.append(ship[X - 1][Y])
                randnum += 1
        if X + 1 < D:
            if isinstance(ship[X + 1][Y], int):
                randselect.append(ship[X + 1][Y])
                randnum += 1
        if Y + 1 < D:
            if isinstance(ship[X][Y + 1], int):
                randselect.append(ship[X][Y + 1])
                randnum += 1
        
        if randnum > 0:
            randnum = random.randrange(randnum)
            deadends.remove(tuple([X, Y]))
            if Y - 1 >= 0:
                if isinstance(ship[X][Y - 1], int):
                    if randnum == 0:
                        ship[X][Y - 1] = str('X')
                    randnum -= 1
            if X - 1 >= 0:
                if isinstance(ship[X - 1][Y], int):
                    if randnum == 0:
                        ship[X - 1][Y] = str('X')
                    randnum -= 1
            if X + 1 < D:
                if isinstance(ship[X + 1][Y], int):
                    if randnum == 0:
                        ship[X + 1][Y] = str('X')
                    randnum -= 1
            if Y + 1 < D:
                if isinstance(ship[X][Y + 1], int):
                    if randnum == 0:
                        ship[X][Y + 1] = str('X')
                    randnum -= 1

    print(tabulate(ship))
    print(deadends)

def markOpen(ship, X, Y, D, options):
    ship[X][Y] = str('X')
    
    if (X, Y) in options:
        options.remove(tuple([X, Y]))

    if Y - 1 >= 0:
        if isinstance(ship[X][Y - 1], int):
            ship[X][Y - 1] += 1 
            if ship[X][Y - 1] == 1:
                options.append(tuple([X, Y - 1]))
            elif ship[X][Y - 1] > 1 and (X, Y - 1) in options:
                options.remove(tuple([X, Y - 1]))
    if X - 1 >= 0:
        if isinstance(ship[X - 1][Y], int):
            ship[X - 1][Y] += 1
            if ship[X - 1][Y] == 1:
                options.append(tuple([X - 1, Y]))
            elif ship[X - 1][Y] > 1 and (X - 1, Y) in options:
                options.remove(tuple([X - 1, Y]))
    if X + 1 < D:
        if isinstance(ship[X + 1][Y], int):
            ship[X + 1][Y] += 1
            if ship[X + 1][Y] == 1:
                options.append(tuple([X + 1, Y]))
            elif ship[X + 1][Y] > 1 and (X + 1, Y) in options:
                options.remove(tuple([X + 1, Y]))
    if Y + 1 < D:
        if isinstance(ship[X][Y + 1], int):
            ship[X][Y + 1] += 1
            if ship[X][Y + 1] == 1:
                options.append(tuple([X, Y + 1]))
            elif ship[X][Y + 1] > 1 and (X, Y + 1) in options:
                options.remove(tuple([X, Y + 1])) 
    
    #print(tabulate(ship))
    #print(options)
    
    return options

def main():
    D = 40
    theShip(D)

if __name__ == "__main__":
    main()

-  -  -  -  -  -  -  -  -  -
X  X  X  X  X  X  X  X  X  0
X  0  0  0  X  0  X  0  0  X
X  X  X  X  0  X  0  X  0  X
X  0  X  0  X  X  0  X  0  X
X  X  0  X  X  0  X  X  X  X
X  0  X  X  0  X  X  0  X  0
X  0  X  0  X  X  0  X  0  X
X  X  X  X  X  0  X  X  X  X
0  X  0  X  0  X  0  X  0  X
X  X  0  X  X  X  X  X  X  0
-  -  -  -  -  -  -  -  -  -
[(2, 7), (6, 7), (0, 8), (4, 1), (1, 6), (7, 6), (8, 9), (9, 0), (2, 5), (3, 2), (2, 3), (1, 9), (1, 4), (6, 9), (5, 8), (9, 8), (8, 5)]
-  -  -  -  -  -  -  -  -  -
X  X  X  X  X  X  X  X  X  X
X  0  0  X  X  X  X  X  0  X
X  X  X  X  0  X  0  X  X  X
X  0  X  X  X  X  0  X  0  X
X  X  0  X  X  0  X  X  X  X
X  0  X  X  0  X  X  0  X  X
X  0  X  0  X  X  0  X  0  X
X  X  X  X  X  0  X  X  X  X
0  X  0  X  0  X  0  X  X  X
X  X  0  X  X  X  X  X  X  0
-  -  -  -  -  -  -  -  -  -
[(6, 7), (4, 1), (7, 6), (9, 0), (3, 2), (1, 9), (6, 9), (9, 8), (8, 5)]
